## Image Processing Using Keras and Tensorflow - Chris Kim

This notebook was adapted from github user [nirmalyaghosh](https://github.com/nirmalyaghosh/deep-learning-vm), and user [fchollet](https://github.com/fchollet/keras/issues/3109). Lastly, most information was adapted from a great website called [Machine Learning Mastery](http://machinelearningmastery.com/).

The code below is my first attempt to use image processing using the tools made available in Python, namely `keras` and `TensorFlow` as the backend. This exploration was inspired by [doyleax](https://doyleax.github.io/Portfolio/capstone.html) and her Capstone project. Side profile images of 1780 pairs of shoes are transformed to be evaluated in a neural network to try to predict whether a shoe is classified as rare or not (determined by a metric composed of retail price and average deadstock price).

All images are taken from [StockX](www.stockx.com).

In [6]:
import numpy as np
import pickle
import matplotlib.pyplot as plt

np.random.seed(7)
%matplotlib inline

In [7]:
with open('shoenumbers.pickle', 'rb') as handle:
    X = pickle.load(handle, encoding='bytes')
with open('targetvars.pickle', 'rb') as handle:
    y = pickle.load(handle, encoding='bytes')

print(np.array(X).shape)
print(np.array(y).shape)

(1877, 200, 280, 3)
(1877,)


In [8]:
baseline = y.mean()
print(baseline)

0.503462972829


In [9]:
# Change size of images so it's not so huge when we pass it through keras
import cv2
new_X = []
for row in range(len(X)):
    new_X.append(cv2.resize(X[row], (140,100)))
np.array(new_X).shape

(1877, 100, 140, 3)

In [10]:
# Prepare data for training and testing
X = np.array(new_X)
X = X.astype('float32')
X /= 255.0
y = np.array(y)

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75)

In [5]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

Using TensorFlow backend.


In [12]:
# Model creation by machinelearningmastery
num_classes=2
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(100,140,3), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [15]:
# Compile the model
epochs = 50
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
# print(model.summary())

In [16]:
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=32)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 1407 samples, validate on 470 samples
Epoch 1/50
1407/1407 [==============================] - 150s - loss: 0.7484 - acc: 0.5167 - val_loss: 0.6984 - val_acc: 0.4468
Epoch 2/50
1407/1407 [==============================] - 157s - loss: 0.6928 - acc: 0.5238 - val_loss: 0.7027 - val_acc: 0.4468
Epoch 3/50
1407/1407 [==============================] - 155s - loss: 0.6893 - acc: 0.5330 - val_loss: 0.6996 - val_acc: 0.4702
Epoch 4/50
1407/1407 [==============================] - 157s - loss: 0.6784 - acc: 0.5714 - val_loss: 0.7001 - val_acc: 0.4957
Epoch 5/50
1407/1407 [==============================] - 154s - loss: 0.6657 - acc: 0.5920 - val_loss: 0.6867 - val_acc: 0.5340
Epoch 6/50
1407/1407 [==============================] - 161s - loss: 0.6324 - acc: 0.6603 - val_loss: 0.6788 - val_acc: 0.5766
Epoch 7/50
1407/1407 [==============================] - 148s - loss: 0.6070 - acc: 0.6681 - val_loss: 0.6849 - val_acc: 0.5809
Epoch 8/50
1407/1407 [==============================] - 139s - l

In [17]:
y_pred = model.predict_classes(X_test)

470/470 [==============================] - 11s    


In [18]:
y_pred

array([0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 1,

## Conclusion
Neural Networking increased the accuracy of my model of baseline: 50% by 20.85%. If I wanted to increase the score, I think the following factors could help increase the accuracy.
- Image transformations
- More enough epochs
- Alter parameters/layers used
- More data